# Intuit Quickbooks Upgrade
* Name: Nameless
* GitLab id:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pyrsm import gains, gains_plot, lift, lift_plot, confusion, profit_max, ROME_max
import copy 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import recall_score
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from operator import itemgetter, attrgetter
import random

Using TensorFlow backend.


In [2]:
## loading the data - this dataset must NOT be changed
intuit75k = pd.read_pickle("../data/intuit75k.pkl")
intuit75k["res1_yes"] = (intuit75k["res1"] == "Yes").astype(int)
intuit75k.head()

,id,zip,zip_bins,sex,bizflag,numords,dollars,last,sincepurch,version1,owntaxprod,upgraded,res1,training,res1_yes
0,1,94553,18,Male,0,2,109.5,5,12,0,0,0,No,1,0
1,2,53190,10,Unknown,0,1,69.5,4,3,0,0,0,No,0,0
2,3,37091,8,Male,0,4,93.0,14,29,0,0,1,No,0,0
3,4,02125,1,Male,0,1,22.0,17,1,0,0,0,No,1,0
4,5,60201,11,Male,0,1,24.5,2,3,0,0,0,No,0,0


In [3]:
## loading the zip code information data
zip_info = pd.read_csv("../data/zip_code.csv")
zip_info.head()

,id,State_Name,ST,Zip_Min,Zip_Max
0,6,Alaska,AK,99501,99950
1,5,Alabama,AL,35004,36925
2,69,Arkansas,AR,71601,72959
3,8,Arkansas (Texarkana),AR,75502,75502
4,7,Arizona,AZ,85001,86556


In [4]:
## sort the dataframe according to zip_max
zip_info = pd.read_csv("../data/zip_code.csv")
zip_info = zip_info.sort_values(by='Zip_Min')
zip_info.head()

,id,State_Name,ST,Zip_Min,Zip_Max
53,44,Puerto Rico,PR,0,0
26,26,Massachusetts,MA,1001,2791
54,45,Rhode Island,RI,2801,2940
42,34,New Hampshire,NH,3031,3897
31,24,Maine,ME,3901,4992


In [5]:
## define a function to match each zip code with different states
def match(zip_code):
    n = 0
    max_list = zip_info['Zip_Max']
    for i in max_list:
        if zip_code > i:
            n += 1
        else:
            n += 0
    return n

In [7]:
## transfer the zip code information into state and save it in the dataframe
zip_c = []
for i in intuit75k["zip"]:
    n = match(int(i))
    if n < len(zip_info):
        zip_c.append(zip_info.iloc[n,2])
    else:
        zip_c.append('Other')
intuit75k['zip_state'] = zip_c
intuit75k.to_csv('intuit75k.csv',index=False)


,id,zip,zip_bins,sex,bizflag,numords,dollars,last,sincepurch,version1,owntaxprod,upgraded,res1,training,res1_yes,zip_state
0,1,94553,18,Male,0,2,109.5,5,12,0,0,0,No,1,0,CA
1,2,53190,10,Unknown,0,1,69.5,4,3,0,0,0,No,0,0,WI
2,3,37091,8,Male,0,4,93.0,14,29,0,0,1,No,0,0,TN
3,4,02125,1,Male,0,1,22.0,17,1,0,0,0,No,1,0,MA
4,5,60201,11,Male,0,1,24.5,2,3,0,0,0,No,0,0,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,74996,28205,6,Male,1,4,211.5,5,15,0,0,0,Yes,1,1,NC
74996,74997,94806,18,Male,0,1,4.5,35,34,1,0,0,No,1,0,CA
74997,74998,72958,13,Female,1,1,54.5,4,19,1,0,0,No,1,0,AR
74998,74999,29464,6,Male,0,1,69.5,20,27,0,0,1,No,1,0,SC


In [4]:
intuit75k= pd.read_csv('../intuit75k_withstate.csv')

In [5]:
## onehot encode the categorical variables, such as 'sex' and 'zip_state'
data = intuit75k
onehot=['sex', 'zip_state']
onehot_encoder=preprocessing.OneHotEncoder(sparse=False)
onehotdata=onehot_encoder.fit_transform(data.loc[:,onehot])
with_onehot = pd.DataFrame(onehotdata)
with_onehot.columns=onehot_encoder.get_feature_names()
with_onehot

,x0_Female,x0_Male,x0_Unknown,x1_AK,x1_AL,x1_AR,x1_AZ,x1_CA,x1_CO,x1_CT,...,x1_SD,x1_TN,x1_TX,x1_UT,x1_VA,x1_VT,x1_WA,x1_WI,x1_WV,x1_WY
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74996,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74997,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
## combine onehot dataframe data and the original data together
combind_data = pd.concat([data[["id",'bizflag', 'numords', 'dollars', 'last','sincepurch', 
                               'version1', 'owntaxprod', 'upgraded', 'res1_yes', 'training']],with_onehot],axis=1)

## split the whole dataset into training and testing set
train = combind_data[combind_data['training'] == 1]
test = combind_data[combind_data['training'] == 0]

## get X and y data
X_train = train.drop(columns=['id', 'training', "res1_yes"])
y_train = train["res1_yes"]
X_test = test.drop(columns=['id', 'training', "res1_yes"])
y_test = test["res1_yes"]

## create evaluation dataframe
eval_dat = test[["id", "res1_yes", "training"]]

In [7]:
# scaling the training data
scaler = StandardScaler()
scaler.fit(X_train)

# apply transformation to training data
Xs_train = scaler.transform(X_train)

In [8]:
# CV for NN
clf = MLPClassifier(
    solver="adam",
    learning_rate_init=0.01,
    alpha=0.01,
    hidden_layer_sizes=(2, 2),
    random_state=1234,
    max_iter=1000,
)

nr_hnodes = range(4, 6)
hls = (
    list(zip(nr_hnodes))
    + list(zip(nr_hnodes, nr_hnodes))
    + list(zip(nr_hnodes, nr_hnodes, nr_hnodes))
)
hls

param_grid = {"hidden_layer_sizes": hls, "alpha": [0.001, 0.01, 0.05]}
scoring = {"AUC": "roc_auc"}

clf_cv = GridSearchCV(
    clf, param_grid, scoring=scoring, cv=5, n_jobs=4, refit="AUC", verbose=5
)
clf_cv.fit(Xs_train, y_train)

clf_cv.best_params_
clf_cv.best_score_
results = pd.DataFrame(clf_cv.cv_results_)
results = results.sort_values(by=["rank_test_AUC"])
results

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   30.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done  90 out of  90 | elapsed:  3.9min finished


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_hidden_layer_sizes,params,split0_test_AUC,split1_test_AUC,split2_test_AUC,split3_test_AUC,split4_test_AUC,mean_test_AUC,std_test_AUC,rank_test_AUC
9,10.876472,0.916870,0.011441,0.000622,0.01,"(5, 5)","{'alpha': 0.01, 'hidden_layer_sizes': (5, 5)}",0.727090,0.753113,0.755997,0.747490,0.752234,0.747185,0.010413,1
12,5.049132,0.428625,0.010579,0.002504,0.05,"(4,)","{'alpha': 0.05, 'hidden_layer_sizes': (4,)}",0.725348,0.752338,0.753383,0.746404,0.757045,0.746904,0.011307,2
15,7.495094,1.485160,0.010877,0.001934,0.05,"(5, 5)","{'alpha': 0.05, 'hidden_layer_sizes': (5, 5)}",0.724620,0.751052,0.758645,0.741981,0.758189,0.746898,0.012674,3
13,5.082394,0.949531,0.010653,0.002337,0.05,"(5,)","{'alpha': 0.05, 'hidden_layer_sizes': (5,)}",0.729727,0.746751,0.754921,0.743607,0.757579,0.746517,0.009833,4
14,8.163492,1.113429,0.008587,0.000531,0.05,"(4, 4)","{'alpha': 0.05, 'hidden_layer_sizes': (4, 4)}",0.723970,0.753023,0.754005,0.746901,0.754273,0.746434,0.011550,5
6,6.516671,1.014691,0.011236,0.001983,0.01,"(4,)","{'alpha': 0.01, 'hidden_layer_sizes': (4,)}",0.720659,0.755619,0.758144,0.741924,0.755181,0.746305,0.014018,6
1,8.590107,1.443866,0.008571,0.000435,0.001,"(5,)","{'alpha': 0.001, 'hidden_layer_sizes': (5,)}",0.723405,0.753397,0.754651,0.745026,0.754988,0.746294,0.012012,7
7,4.952278,0.780762,0.009391,0.001360,0.01,"(5,)","{'alpha': 0.01, 'hidden_layer_sizes': (5,)}",0.729248,0.750043,0.749876,0.748037,0.752928,0.746026,0.008534,8
16,10.626315,1.672921,0.011303,0.001601,0.05,"(4, 4, 4)","{'alpha': 0.05, 'hidden_layer_sizes': (4, 4, 4)}",0.728257,0.752876,0.756351,0.742845,0.748770,0.745820,0.009866,9
11,14.826287,2.206502,0.010828,0.002042,0.01,"(5, 5, 5)","{'alpha': 0.01, 'hidden_layer_sizes': (5, 5, 5)}",0.718181,0.746787,0.755812,0.750731,0.756599,0.745622,0.014176,10


From the results above, the best parameters are `alpha = 0.01`, `hidden_layer_sizes = (5, 5)`

In [ ]:
## set seed before running for loop
random.seed(1234)

## generate 100 bootstraps
for i in range(100):
    sample_train = train.sample(n = 52500, replace= False)
    X_sample_train = sample_train.drop(columns=['id', 'training', "res1_yes"])
    y_sample_train = sample_train["res1_yes"]

    # scaling the training data
    scaler = StandardScaler()
    scaler.fit(X_sample_train)

    # apply transformation to training data
    Xs_sample_train = scaler.transform(X_sample_train)
    Xs_train = scaler.transform(X_train)

    # apply same transformation to test data
    Xs_test = scaler.transform(X_test)

    # combining so we can predict for the full dataset as well
    Xs = np.concatenate((Xs_train, Xs_test), axis=0)
    # CV for NN
    nr_hnodes = [5]
    hls = (
        list(zip(nr_hnodes))
        + list(zip(nr_hnodes, nr_hnodes))
        + list(zip(nr_hnodes, nr_hnodes, nr_hnodes))
    )

    param_grid = {"hidden_layer_sizes": hls, "alpha": [0.001]}
    scoring = {"AUC": "roc_auc"}

    clf_cv = GridSearchCV(
        clf, param_grid, scoring=scoring, cv=5, n_jobs=4, refit="AUC", verbose=5
    )
    clf_cv.fit(Xs_sample_train, y_sample_train)

    clf_cv.best_params_
    clf_cv.best_score_
    results = pd.DataFrame(clf_cv.cv_results_)
    results = results.sort_values(by=["rank_test_AUC"])

    nn_proba = clf_cv.best_estimator_.predict_proba(Xs_test)
    eval_dat['bootstrap'+ str(i+1)] = nn_proba[:, 1]

## save the bootstrap dataframe
eval_dat.to_csv('bootstrap.csv',index=False)

In [16]:
## reload the bootstrap dataframe
eval_dat = pd.read_csv("bootstrap.csv")
bootstrap = eval_dat.drop(columns=['res1_yes', 'training'])
bootstrap.set_index(["id"], inplace=True)
bootstrap

,bootstrap1,bootstrap2,bootstrap3,bootstrap4,bootstrap5,bootstrap6,bootstrap7,bootstrap8,bootstrap9,bootstrap10,...,bootstrap91,bootstrap92,bootstrap93,bootstrap94,bootstrap95,bootstrap96,bootstrap97,bootstrap98,bootstrap99,bootstrap100
id,,,,,,,,,,,,,,,,,,,,,
2,0.037860,0.025047,0.039045,0.032288,0.022090,0.033635,0.021315,0.019967,0.040484,0.023176,...,0.036736,0.025040,0.035509,0.040552,0.030656,0.038241,0.026144,0.043447,0.027927,0.031598
3,0.081811,0.084028,0.059387,0.091529,0.080492,0.093445,0.095693,0.082799,0.088256,0.051732,...,0.077767,0.100047,0.081867,0.074834,0.085107,0.102296,0.070426,0.093935,0.055287,0.078486
5,0.016932,0.017646,0.013097,0.014982,0.015625,0.017290,0.015903,0.010762,0.025401,0.019429,...,0.010326,0.020143,0.014854,0.022715,0.012642,0.022807,0.013726,0.029569,0.020703,0.018522
7,0.026680,0.024929,0.031298,0.019075,0.021511,0.029724,0.023149,0.019021,0.030076,0.026986,...,0.027337,0.020952,0.016081,0.030080,0.017394,0.021835,0.021834,0.033954,0.023110,0.024411
9,0.008636,0.015885,0.011020,0.008052,0.008476,0.011624,0.008712,0.008576,0.010047,0.010224,...,0.007461,0.008927,0.008564,0.013852,0.009387,0.015455,0.010731,0.011619,0.013018,0.010829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74982,0.027599,0.033909,0.028948,0.026024,0.025638,0.026555,0.022141,0.025164,0.021861,0.024828,...,0.021937,0.026699,0.020981,0.034965,0.021324,0.036721,0.024589,0.024059,0.022638,0.034871
74983,0.023466,0.012611,0.009703,0.017881,0.008100,0.013567,0.006248,0.006008,0.014736,0.010633,...,0.015529,0.011679,0.011479,0.015452,0.010798,0.013759,0.009231,0.014006,0.013666,0.019231
74988,0.108780,0.162261,0.091277,0.121059,0.149009,0.166899,0.136483,0.105190,0.112783,0.147538,...,0.114300,0.128213,0.112840,0.115516,0.096589,0.104583,0.130469,0.135980,0.185786,0.134314


In [18]:
## get the lower bound for test set from the bootstraps
lb = []
for i in range(len(eval_dat)):
    l = bootstrap.iloc[i,:].quantile(0.05)
    lb.append(l)
eval_dat['lb'] = lb
eval_dat['lb']

0        0.021247
1        0.059133
2        0.010417
3        0.016187
4        0.007200
           ...   
22495    0.020976
22496    0.005525
22497    0.089751
22498    0.044221
22499    0.049438
Name: lb, Length: 22500, dtype: float64

In [10]:
# scaling the training data
scaler = StandardScaler()
scaler.fit(X_train)

# apply transformation to training data
Xs_train = scaler.transform(X_train)

# apply same transformation to test data
Xs_test = scaler.transform(X_test)

In [21]:
# CV for NN
nr_hnodes = [5]
hls = (
    list(zip(nr_hnodes))
    + list(zip(nr_hnodes, nr_hnodes))
    + list(zip(nr_hnodes, nr_hnodes, nr_hnodes))
)

param_grid = {"hidden_layer_sizes": hls, "alpha": [0.01]}
scoring = {"AUC": "roc_auc"}

clf_cv = GridSearchCV(
    clf, param_grid, scoring=scoring, cv=5, n_jobs=4, refit="AUC", verbose=5
)
clf_cv.fit(Xs_train, y_train)

# prediction on test set
pred = clf_cv.predict_proba(Xs_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, pred[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  12 out of  15 | elapsed:   14.9s remaining:    3.7s
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:   20.0s finished


0.7374105706432863

In [19]:
# prediction on test set
pred = clf_cv.predict_proba(Xs_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, pred[:, 1])
auc_rf = metrics.auc(fpr, tpr)
eval_dat['cvnn']=pred[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [21]:
profit = profit_max(
    eval_dat.query("training == 0"), 'res1_yes', 1, "cvnn", cost=1.41, margin=60
)
profit

36972.75

In [22]:
# calculate the key metrics of the model
auc_rf = metrics.auc(fpr, tpr)
profit = profit_max(
    eval_dat.query("training == 0"), 'res1_yes', 1, "lb", cost=1.41, margin=60
)
ROME = ROME_max(eval_dat.query("training == 0"), "res1_yes", 1, "lb", cost=1.41, margin=60)



In [28]:
eva = pd.DataFrame({'auc':[auc_rf], 'profit':[profit], 'ROME':[ROME]})
eva

,auc,profit,ROME
0,0.737411,36271.38,2.741887
